In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.util import everygrams

np.random.seed(42)
random.seed(42)

In [2]:
#загружаем словарь сильных и слабых слов, где есть биграммы

dictionary = pd.read_excel('dict_ngrams.xlsx')
dictionary

#в результате мы не использовали сильные слова, но код для них оставлю здесь на всякий случай

,weak,strong
0,почти,определенно
1,практически,явно
2,приблизительно,явный
3,приближенно,однозначно
4,видимо,отчетливо
5,вероятно,точно
6,похоже,наибольший
7,кажется,наименьший
8,предполагаемый,должен
9,неопределенный,никогда


In [3]:
#преобразуем колонки в списки

weak = dictionary['weak'].tolist()
strong = dictionary['strong'].tolist()

In [4]:
m = MorphAnalyzer()
regex = re.compile("[А-Яа-я]+")

#очищаем текст + токенизация
def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

#лемматизация токена
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

#общая функция 
def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    return lemmas

Работаем с дата-сетом ПРЕСС-РЕЛИЗОВ по ставке

In [5]:
#загружаем тексты пресс-релизов по ставке

pr = pd.read_excel('!PR_datasetN4.xls')
pr.head()

,Date,Text
0,2015-01-30,Совет директоров Банка России 30 января 2015 г...
1,2015-03-13,Совет директоров Банка России 13 марта 2015 го...
2,2015-04-30,Совет директоров Банка России 30 апреля 2015 г...
3,2015-06-15,Совет директоров Банка России 15 июня 2015 год...
4,2015-07-31,Совет директоров Банка России 31 июля 2015 год...


In [6]:
#лемматизируем пресс-релизы

lemmas = list(pr['Text'].apply(clean_text))
pr['lemmas'] = lemmas
pr.head()

,Date,Text,lemmas
0,2015-01-30,Совет директоров Банка России 30 января 2015 г...,"[совет, директор, банк, россия, январь, год, п..."
1,2015-03-13,Совет директоров Банка России 13 марта 2015 го...,"[совет, директор, банк, россия, март, год, при..."
2,2015-04-30,Совет директоров Банка России 30 апреля 2015 г...,"[совет, директор, банк, россия, апрель, год, п..."
3,2015-06-15,Совет директоров Банка России 15 июня 2015 год...,"[совет, директор, банк, россия, июнь, год, при..."
4,2015-07-31,Совет директоров Банка России 31 июля 2015 год...,"[совет, директор, банк, россия, июль, год, при..."


In [7]:
#функция преобразования tuple в str (понадобится дальше)

def convertTuple(tup):
    st = ' '.join([str(value) for value in tup])
    return st

In [8]:
#преобразуем леммы в униграммы и биграммы

pr['ngrams'] = pr['lemmas'].apply(lambda row: list(everygrams(row, max_len=2)))
for i, row in pr.iterrows():
    pr.at[i,'ngrams'] = [convertTuple(tup) for tup in pr.at[i,'ngrams']]
pr.head()

,Date,Text,lemmas,ngrams
0,2015-01-30,Совет директоров Банка России 30 января 2015 г...,"[совет, директор, банк, россия, январь, год, п...","[совет, директор, банк, россия, январь, год, п..."
1,2015-03-13,Совет директоров Банка России 13 марта 2015 го...,"[совет, директор, банк, россия, март, год, при...","[совет, директор, банк, россия, март, год, при..."
2,2015-04-30,Совет директоров Банка России 30 апреля 2015 г...,"[совет, директор, банк, россия, апрель, год, п...","[совет, директор, банк, россия, апрель, год, п..."
3,2015-06-15,Совет директоров Банка России 15 июня 2015 год...,"[совет, директор, банк, россия, июнь, год, при...","[совет, директор, банк, россия, июнь, год, при..."
4,2015-07-31,Совет директоров Банка России 31 июля 2015 год...,"[совет, директор, банк, россия, июль, год, при...","[совет, директор, банк, россия, июль, год, при..."


In [9]:
#создадим списки с количеством слабых и сильных слов, которые встретились в пресс-релизах

weak_pr, strong_pr = [], []
count_weak, count_strong = 0, 0

for row in pr['ngrams']:
    for word in row:
        if word in weak:
            count_weak += 1
        elif word in strong:
            count_strong += 1
    weak_pr.append(count_weak)
    strong_pr.append(count_strong)
    count_weak, count_strong = 0, 0
    
print(weak_pr)
print(strong_pr)

[3, 1, 3, 3, 2, 1, 1, 4, 2, 3, 1, 1, 2, 2, 2, 3, 2, 3, 5, 1, 1, 3, 0, 5, 5, 3, 4, 6, 3, 5, 4, 5, 4, 4, 5, 3, 2, 4, 5, 5, 5, 2, 3, 5, 2, 9, 9, 4, 5, 3, 2, 3, 4, 5, 3, 1, 1, 0, 0, 0, 2, 1, 1, 0, 3, 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
print('Всего слабых слов в пресс-релизах:', sum(weak_pr))
print('Всего сильных слов в пресс-релизах:', sum(strong_pr))

Всего слабых слов в пресс-релизах: 200
Всего сильных слов в пресс-релизах: 1


In [14]:
pr['weak'] = weak_pr
pr['strong'] = strong_pr
pr

,Date,Text,lemmas,ngrams,weak,strong
0,2015-01-30,Совет директоров Банка России 30 января 2015 г...,"[совет, директор, банк, россия, январь, год, п...","[совет, директор, банк, россия, январь, год, п...",3,0
1,2015-03-13,Совет директоров Банка России 13 марта 2015 го...,"[совет, директор, банк, россия, март, год, при...","[совет, директор, банк, россия, март, год, при...",1,0
2,2015-04-30,Совет директоров Банка России 30 апреля 2015 г...,"[совет, директор, банк, россия, апрель, год, п...","[совет, директор, банк, россия, апрель, год, п...",3,0
3,2015-06-15,Совет директоров Банка России 15 июня 2015 год...,"[совет, директор, банк, россия, июнь, год, при...","[совет, директор, банк, россия, июнь, год, при...",3,0
4,2015-07-31,Совет директоров Банка России 31 июля 2015 год...,"[совет, директор, банк, россия, июль, год, при...","[совет, директор, банк, россия, июль, год, при...",2,0
...,...,...,...,...,...,...
61,2022-05-26,Совет директоров Банка России принял решение с...,"[совет, директор, банк, россия, принять, решен...","[совет, директор, банк, россия, принять, решен...",1,0
62,2022-06-10,Совет директоров Банка России 10 июня 2022 год...,"[совет, директор, банк, россия, июнь, год, при...","[совет, директор, банк, россия, июнь, год, при...",1,0
63,2022-07-22,Совет директоров Банка России 22 июля 2022 год...,"[совет, директор, банк, россия, июль, год, при...","[совет, директор, банк, россия, июль, год, при...",0,0
64,2022-09-16,Совет директоров Банка России 16 сентября 2022...,"[совет, директор, банк, россия, сентябрь, год,...","[совет, директор, банк, россия, сентябрь, год,...",3,0


In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
weak = pr['weak'].to_numpy()
weak = weak.reshape(-1, 1)
scaler.fit(weak)
print(scaler.data_max_)

[9.]


In [24]:
pr['tone'] = scaler.transform(weak)

,Date,Text,lemmas,ngrams,weak,strong,tone
0,2015-01-30,Совет директоров Банка России 30 января 2015 г...,"[совет, директор, банк, россия, январь, год, п...","[совет, директор, банк, россия, январь, год, п...",3,0,0.333333
1,2015-03-13,Совет директоров Банка России 13 марта 2015 го...,"[совет, директор, банк, россия, март, год, при...","[совет, директор, банк, россия, март, год, при...",1,0,0.111111
2,2015-04-30,Совет директоров Банка России 30 апреля 2015 г...,"[совет, директор, банк, россия, апрель, год, п...","[совет, директор, банк, россия, апрель, год, п...",3,0,0.333333
3,2015-06-15,Совет директоров Банка России 15 июня 2015 год...,"[совет, директор, банк, россия, июнь, год, при...","[совет, директор, банк, россия, июнь, год, при...",3,0,0.333333
4,2015-07-31,Совет директоров Банка России 31 июля 2015 год...,"[совет, директор, банк, россия, июль, год, при...","[совет, директор, банк, россия, июль, год, при...",2,0,0.222222
...,...,...,...,...,...,...,...
61,2022-05-26,Совет директоров Банка России принял решение с...,"[совет, директор, банк, россия, принять, решен...","[совет, директор, банк, россия, принять, решен...",1,0,0.111111
62,2022-06-10,Совет директоров Банка России 10 июня 2022 год...,"[совет, директор, банк, россия, июнь, год, при...","[совет, директор, банк, россия, июнь, год, при...",1,0,0.111111
63,2022-07-22,Совет директоров Банка России 22 июля 2022 год...,"[совет, директор, банк, россия, июль, год, при...","[совет, директор, банк, россия, июль, год, при...",0,0,0.000000
64,2022-09-16,Совет директоров Банка России 16 сентября 2022...,"[совет, директор, банк, россия, сентябрь, год,...","[совет, директор, банк, россия, сентябрь, год,...",3,0,0.333333


In [31]:
pr['tone'].isnull().any()

False

In [32]:
pr_tone = pr[['Date','Text', 'tone']].copy()

In [33]:
pr_tone.to_excel('pr_tone.xls')

Работаем с датасетом ЗАЯВЛЕНИЙ по ставке

In [19]:
#загружаем датасет заявлений по ставке

zp = pd.read_excel('!ZP_DATASET.xls')
zp.head()

,Date,Text
0,2015-03-13,Сегодня Совет директоров Банка России принял р...
1,2015-06-15,Сегодня Совет директоров Банка России принял р...
2,2015-09-11,Сегодня Банк России принял решение сохранить к...
3,2015-12-11,Сегодня Банк России принял решение сохранить к...
4,2016-03-18,Сегодня Банк России принял решение сохранить к...


In [20]:
#лемматизируем заявления

lemmas = list(zp['Text'].apply(clean_text))
zp['lemmas'] = lemmas
zp.head()

,Date,Text,lemmas
0,2015-03-13,Сегодня Совет директоров Банка России принял р...,"[сегодня, совет, директор, банк, россия, приня..."
1,2015-06-15,Сегодня Совет директоров Банка России принял р...,"[сегодня, совет, директор, банк, россия, приня..."
2,2015-09-11,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр..."
3,2015-12-11,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр..."
4,2016-03-18,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр..."


In [21]:
#создаем униграммы и биграммы

zp['ngrams'] = zp['lemmas'].apply(lambda row: list(everygrams(row, max_len=2)))
for i, row in zp.iterrows():
    zp.at[i,'ngrams'] = [convertTuple(tup) for tup in zp.at[i,'ngrams']]
zp.head()

,Date,Text,lemmas,ngrams
0,2015-03-13,Сегодня Совет директоров Банка России принял р...,"[сегодня, совет, директор, банк, россия, приня...","[сегодня, совет, директор, банк, россия, приня..."
1,2015-06-15,Сегодня Совет директоров Банка России принял р...,"[сегодня, совет, директор, банк, россия, приня...","[сегодня, совет, директор, банк, россия, приня..."
2,2015-09-11,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр...","[сегодня, банк, россия, принять, решение, сохр..."
3,2015-12-11,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр...","[сегодня, банк, россия, принять, решение, сохр..."
4,2016-03-18,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр...","[сегодня, банк, россия, принять, решение, сохр..."


In [22]:
#создадим списки с количеством слабых и сильных слов, которые встретились в заявлениях по ставке

weak_zp, strong_zp = [], []
count_weak, count_strong = 0, 0

for row in zp['ngrams']:
    for word in row:
        if word in weak:
            count_weak += 1
        elif word in strong:
            count_strong += 1
    weak_zp.append(count_weak)
    strong_zp.append(count_strong)
    count_weak, count_strong = 0, 0
    
print(weak_zp)
print(strong_zp)

[5, 12, 12, 14, 16, 8, 8, 7, 10, 9, 8, 10, 5, 9, 4, 7, 10, 8, 4, 7, 12, 3, 6, 8, 3, 7, 12, 3, 5, 4, 3, 2, 7, 7, 3, 5, 3, 2, 3, 5, 8, 6, 7, 7]
[0, 1, 1, 1, 0, 1, 1, 2, 2, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0, 2, 0, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0, 2, 1, 2, 1]


In [23]:
print('Всего слабых слов в заявлениях:', sum(weak_zp))
print('Всего сильных слов в заявлениях:', sum(strong_zp))

Всего слабых слов в заявлениях: 304
Всего сильных слов в заявлениях: 34


In [24]:
zp['weak'] = weak_zp
zp['strong'] = strong_zp

In [25]:
zp['formulaic'] = (zp['weak']-zp['strong'])/(zp['weak']+zp['strong'])

In [26]:
zp['formulaic'] = zp['formulaic'].fillna(0)

In [27]:
zp_formulaic = zp[['Date','Text', 'formulaic']].copy()

In [28]:
zp_formulaic.to_excel('zp_formulaic_ngrams.xls')